In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import os
import os.path as osp
import json

In [30]:
# sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
path = '/nfs/students/fuchsgru/artifacts/week10-sentence-transformers-paraphrase-multilingual-MiniLM-L12-v2/model_no_remove_gcn_hidden_sizes_[64-32]_weight_scale_0.9/2-2-0'

# bert-base-uncased
#path = '/nfs/students/fuchsgru/artifacts/week10_bert-base-uncased/model_no_remove_gcn_hidden_sizes_[64-32]_weight_scale_0.9/4-4-0'

# allenai/longformer-base-4096
#path = '/nfs/students/fuchsgru/artifacts/week10-allenai-longformer-base-4096/model_no_remove_gcn_hidden_sizes_[64-32]_weight_scale_0.9/2-2-0'

with open(osp.join(path, 'metrics.json'))as f:
    results = json.load(f)

In [31]:
data = {}
for k, v in results.items():
    mean = np.mean(v)
    std = np.std(v)
    if k.startswith('val'):
        continue
    if '-no-edges' in k:
        k = k.replace('-no-edges', '')
        no_edges = True
    else:
        no_edges = False
        
    tokens = k.split('_')
    ood_setting, tokens = tokens[-1], tokens[:-1]
    
    if tokens[0] == 'accuracy' and tokens[1] in ('id', 'ood'):
        metric, tokens = '-'.join(tokens[:2]), tokens[2:]
    else:
        metric, tokens = tokens[0], tokens[1:]
        
    if metric in ('auroc', 'aucpr'):
        type_ = '-'.join(tokens)
        if '-relative' in type_:
            type_ = type_.replace('-relative', '')
            relative = 'relative'
        else:
            relative = ''
        if '-max' in type_:
            type_ = type_.replace('-max', '')
            density_mode = 'max'
        elif '-weighted' in type_:
            type_ = type_.replace('-weighted', '')
            density_mode = 'weighted'
        else:
            density_mode = ''
        
    else:
        type_, relative, density_mode = '', '', ''
    
    if 'aucpr' in metric:
        continue
        
    data_key = tuple(str(k).replace('-', ' ') for k in (no_edges, metric, type_, relative, density_mode, ood_setting,))
    
    data[data_key + ('mean',)] = [mean]
    data[data_key + ('std',)] = [std]
    

df = pd.DataFrame(data).T.sort_index().reset_index((-2, 0, -1))



In [32]:

subdfs = []
for n, g in df.groupby(['level_5', 'level_0', 'level_6']):
    g = pd.DataFrame(g[0])
    g.columns = [n]
    subdfs.append(g)
df_cat = pd.concat(subdfs, axis=1)

idx = pd.MultiIndex.from_tuples(df_cat.columns, names=(
    'OOD Setting', 'Remove Edges', 'Stat'
))
df_cat.columns=idx
pd.set_option("display.precision", 2)
df_cat

OOD Setting                                               ber              \
Remove Edges                                            False        True   
Stat                                                     mean   std  mean   
accuracy                                                 0.80  0.03  0.60   
accuracy id                                              0.81  0.03  0.65   
accuracy ood                                             0.72  0.10  0.17   
auroc        gpc diag:no                       max       0.78  0.11  0.96   
                                               weighted  0.78  0.11  0.96   
                                      relative max       0.76  0.11  0.87   
                                               weighted  0.76  0.11  0.87   
             logit energy                                0.70  0.09  0.96   
             max score                                   0.70  0.09  0.99   
             total predictive entropy                    0.71  0.09  1.00   
ece                                                      0.10  0.03  0.10   

OOD Setting                                                         loc        \
Remove Edges                                                      False         
Stat                                                          std  mean   std   
accuracy                                                 3.26e-02  0.81  0.03   
accuracy id                                              4.01e-02  0.81  0.03   
accuracy ood                                             8.32e-02   NaN   NaN   
auroc        gpc diag:no                       max       2.91e-02  0.71  0.03   
                                               weighted  2.91e-02  0.71  0.03   
                                      relative max       1.15e-01  0.79  0.03   
                                               weighted  1.13e-01  0.79  0.03   
             logit energy                                3.53e-02  0.78  0.06   
             max score                                   2.29e-02  0.79  0.04   
             total predictive entropy                    7.13e-03  0.80  0.04   
ece                                                      2.64e-02  0.10  0.03   

OOD Setting                                                         normal  \
Remove Edges                                             True        False   
Stat                                                     mean   std   mean   
accuracy                                                 0.65  0.03   0.77   
accuracy id                                              0.65  0.03   0.80   
accuracy ood                                              NaN   NaN   0.52   
auroc        gpc diag:no                       max       0.44  0.04   0.89   
                                               weighted  0.44  0.04   0.89   
                                      relative max       0.64  0.05   0.75   
                                               weighted  0.64  0.05   0.75   
             logit energy                                0.75  0.05   0.58   
             max score                                   0.72  0.03   0.66   
             total predictive entropy                    0.73  0.04   0.64   
ece                                                      0.10  0.03   0.09   

OOD Setting                                                                
Remove Edges                                                   True        
Stat                                                      std  mean   std  
accuracy                                                 0.02  0.59  0.03  
accuracy id                                              0.03  0.64  0.03  
accuracy ood                                             0.12  0.13  0.06  
auroc        gpc diag:no                       max       0.03  0.95  0.05  
                                               weighted  0.03  0.95  0.05  
                                      relative max       0.12  0.60  0.15  
         